<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/PAPER_GDRNet_ELECTRICITY_Step4-VAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
from numpy import array
from numpy import hstack
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, MultiTaskLassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt
from sklearn.linear_model import LinearRegression
from keras.models import Model
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import acf
from sklearn.ensemble import RandomForestRegressor
from numpy import arange
#-----------------------------------------------------GET DATA - X, Predicted Wndow from MLP, and Derived Labels, for the entire dataset that was gennerated through VAE-----------------------------------


#-----------for test dataset-----------------------------------------------------------------------
y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/ELEC_predicted_window.csv')
y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/ELEC_derived_window_label.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/ELEC_test_data.csv')


n_features = 6
window_size = int(x_orig.shape[1]/n_features)
x = x_orig.reshape((x_orig.shape[0],window_size,n_features))

sample_size = 1000
#-----------------------------------------------CASES - pot window values and pick a few fixed window sizes -------------------------------------------



plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Distribution before Transformation", fontsize=15)
sns.histplot(y_true, kde=True, color="red")
plt.subplot(1,2,2)

# Import libraries
import matplotlib.pyplot as plt
import numpy as np
fig = plt.figure(figsize =(10, 7))
# Creating plot
plt.boxplot(y_pred)
# show plot
plt.show()

fx_window1 = 5
fx_window2 = 10
fx_window3 = 15
fx_window4 = 20


pred_step = n_future=1



rmse_v1 = list()
rmse_v2 = list()
rmse_v3 = list()
rmse_v4 = list()
rmse_vx = list()





for i in range(100) :
  k=fx_window1
  x1 = x[i]
  df = pd.DataFrame(x1, columns=['V1','V2','V3','V4','V5','V6'])
  df_train, df_test = df[0:-pred_step], df[-pred_step:]
  model= VAR(df_train)
  try:
    model_fitted1 = model.fit(k)
    forecast_input1 = df_train.values[-k:]
    fc1 = model_fitted1.forecast(y=forecast_input1, steps=n_future)
    df_forecast1 = pd.DataFrame(fc1, index=df.index[-n_future:], columns=df.columns)
    mse =  mean_squared_error(df_test['V1'], df_forecast1['V1'].values)
    rmse_v1.append(mse)
  except:
    rmse_v1.append(99999)
    print('VAR could not solve row number')
    print(i, k)

  k=fx_window2
  try:
    model_fitted2 = model.fit(k)
    forecast_input2 = df_train.values[-k:]
    fc2 = model_fitted2.forecast(y=forecast_input2, steps=n_future)
    df_forecast2 = pd.DataFrame(fc2, index=df.index[-n_future:], columns=df.columns)
    mse =  mean_squared_error(df_test['V1'], df_forecast2['V1'].values)
    rmse_v2.append(mse)
  except:
    rmse_v2.append(99999)
    print('VAR could not solve row number')
    print(i, k)

  k=fx_window3
  try:
    model_fitted3 = model.fit(k)
    forecast_input3 = df_train.values[-k:]
    fc3 = model_fitted3.forecast(y=forecast_input3, steps=n_future)
    df_forecast3 = pd.DataFrame(fc3, index=df.index[-n_future:], columns=df.columns)
    mse =  mean_squared_error(df_test['V1'], df_forecast3['V1'].values)
    rmse_v3.append(mse)
  except:
    rmse_v3.append(99999)
    print('VAR could not solve row number')
    print(i, k)

  k=fx_window4
  try:
    model_fitted4 = model.fit(k)
    forecast_input4 = df_train.values[-k:]
    fc4 = model_fitted4.forecast(y=forecast_input4, steps=n_future)
    df_forecast4 = pd.DataFrame(fc4, index=df.index[-n_future:], columns=df.columns)
    mse =  mean_squared_error(df_test['V1'], df_forecast4['V1'].values)
    rmse_v4.append(mse)

  except:
    rmse_v4.append(99999)
    print('VAR could not solve row number')
    print(i, k)


  k=int(y_pred[i])
  try:
    model_fittedx = model.fit(k)
    forecast_inputx = df_train.values[-k:]
    fcx = model_fittedx.forecast(y=forecast_inputx, steps=n_future)
    df_forecastx = pd.DataFrame(fcx, index=df.index[-n_future:], columns=df.columns)
    mse =  mean_squared_error(df_test['V1'], df_forecastx['V1'].values)
    rmse_vx.append(mse)

  except:
    rmse_vx.append(99999)
    print('VAR could not solve row number')
    print(i, k)


print(np.mean(rmse_v1), np.mean(rmse_v2),  np.mean(rmse_v3), np.mean(rmse_v4),  np.mean(rmse_vx))




df.to_csv('/content/drive/MyDrive/PHD/2024/PREDICTION_COMPARISON/Electricity_mse_comparison.csv', index = False)


0.047358956923132384 0.006459317730748585 0.00852711995347731 0.0032252076150122867 0.007152076120046911


In [ ]:
from google.colab import drive
drive.mount('/content/drive')